In [2]:
import pdfplumber
import json

def extract_ttas_tables(pdf_path):
    all_content = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # 提取表格數據
            tables = page.extract_tables()
            # 提取當頁純文字（用於獲取標題與備註）
            text = page.extract_text()
            all_content.append({"text": text, "tables": tables})
    return all_content

In [3]:
extract_ttas_tables(r"C:\Users\ygz08\Work\TTAS\code\rag_files\急診五級檢傷分類基準修正版-總表.pdf")

[{'text': '急診五級檢傷分類基準-修正版\n表一、台灣急診檢傷急迫度分級量表成人首要調節變數標準-生命徵象\nTTAS 級數 一級 二級 三級 四級 五級\n判定依據 復甦急救 危急 緊急 次緊急 非緊急\n◎生命徵象(Vital Signs)\n一、呼吸\n重度呼吸窘迫：呼吸衰竭，過度的中度呼吸窘迫：呼吸費力，呼吸工輕度呼吸窘迫：呼吸困難，心跳過\n綜合描述 呼吸工作而產生疲憊現象，明顯發作增加，使用呼吸輔助肌。 速，在走動時有呼吸急促的現象，\n紺及意識混亂。 沒有明顯呼吸工作的增加。\n說話 單一字或無法言語 片語或不成句 可使用句子\n明顯或惡化的喘鳴呼吸聲，但呼吸\n呼吸道 上氣道阻塞 呼吸道暢通\n道仍暢通。\n呼吸次數 <10 次/分\n血氧飽和度(SpO ) <90% <92% 92%-94%\n2\n尖峰呼氣流速預估值\n<40% 40-60% >60%\n(PEFR predicted)\n備註：\n1.分級判定依據以描述性定義綜合判斷為原則，不以SpO 或其他絕對值為單一分級標準。\n2\n2. SpO 不適合用於長期缺氧、COPD 病人。\n2\n3.若檢傷站有尖峰呼氣流速計，可用來測定病人氣流限制的程度，以評估氣喘的嚴重程度。\n1',
  'tables': [[['TTAS 級數\n判定依據', '一級', '二級', '三級', '四級', '五級'],
    [None, '復甦急救', '危急', '緊急', '次緊急', '非緊急'],
    ['◎生命徵象(Vital Signs)', None, None, None, None, None],
    ['一、呼吸', None, None, None, None, None],
    ['綜合描述',
     '重度呼吸窘迫：呼吸衰竭，過度的\n呼吸工作而產生疲憊現象，明顯發\n紺及意識混亂。',
     '中度呼吸窘迫：呼吸費力，呼吸工\n作增加，使用呼吸輔助肌。',
     '輕度呼吸窘迫：呼吸困難，心跳過\n速，在走動時有呼吸急促的現象，\n沒有明顯呼吸工作的增加。',
     '',
     ''],
    ['說話', '單一字或無法言語', '片語或不成句', '可使用句子', '', ''],
    ['呼吸道', '上氣道阻塞

In [10]:
import pandas as pd
a= pd.read_csv(r"C:\Users\ygz08\Work\TTAS\data\0128\data.csv")
print(a.iloc[0])

檢傷編號                               226040
急診日期                             20251001
生日                               20120329
性別                                      M
體溫                                     37
體重                                       
收縮壓                                   110
舒張壓                                    70
脈搏                                    100
呼吸                                     18
SAO2                                   99
GCS_E                                   4
GCS_V                                   5
GCS_M                                   6
瞳孔左                                      
瞳孔右                                      
身高                                       
LMP                                      
病人主訴                     雙下肢疼痛、入急診就診未小便故入
藥物過敏史                                  無;
檢傷分級                                    3
檢傷大分類英文名                    Genitourinary
檢傷大分類中文名                          A06泌尿系統
檢傷主訴                             A

In [15]:
from datetime import datetime

def format_patient_info(row):
    # 1. 計算年齡與判定對象 (未滿 18 歲為兒童) [cite: 126, 172]
    visit_date = datetime.strptime(str(row['急診日期']), '%Y%m%d')
    birth_date = datetime.strptime(str(row['生日']), '%Y%m%d')
    age = visit_date.year - birth_date.year - ((visit_date.month, visit_date.day) < (birth_date.month, birth_date.day))
    target_group = "兒童" if age < 18 else "成人"

    # 2. 計算 MAP (平均動脈壓) [cite: 25]
    try:
        sbp = float(row['收縮壓'])
        dbp = float(row['舒張壓'])
        map_val = round((sbp - dbp) / 3 + dbp, 1)
    except:
        map_val = "未知"

    # 3. 計算 GCS 總分 
    try:
        gcs_sum = int(row['GCS_E']) + int(row['GCS_V']) + int(row['GCS_M'])
    except:
        gcs_sum = "未知"

    # 4. 組合成結構化文字
    #    - 檢傷主訴分類：{row['檢傷主訴']}
    info_str = f"""
    【病患基本資料】
    - 對象：{target_group} (年齡: {age}歲, 性別: {row['性別']})
    - 主訴：{row['病人主訴']}


    【生理數據 (Vital Signs)】
    - 體溫：{row['體溫']}°C
    - 血壓：{row['收縮壓']}/{row['舒張壓']} mmHg (MAP: {map_val})
    - 心跳/脈搏：{row['脈搏']} 次/分
    - 呼吸：{row['呼吸']} 次/分
    - 血氧 (SaO2)：{row['SAO2']}%
    - 意識狀態 (GCS)：{gcs_sum} 分 (E{row['GCS_E']}, V{row['GCS_V']}, M{row['GCS_M']})
    - 藥物過敏史：{row['藥物過敏史']}
    """
    return info_str, target_group

# 使用範例
row = a.iloc[0]
patient_text, target = format_patient_info(row)

# 接下來將 patient_text 丟入 LLM，並用 target 來過濾 ChromaDB
print(patient_text)


    【病患基本資料】
    - 對象：兒童 (年齡: 13歲, 性別: M)
    - 主訴：雙下肢疼痛、入急診就診未小便故入


    【生理數據 (Vital Signs)】
    - 體溫：37°C
    - 血壓：110/70 mmHg (MAP: 83.3)
    - 心跳/脈搏：100 次/分
    - 呼吸：18 次/分
    - 血氧 (SaO2)：99%
    - 意識狀態 (GCS)：15 分 (E4, V5, M6)
    - 藥物過敏史：無;
    
